In [155]:
# =========================
# CELL 0 — Imports & Config
# =========================
import os, re, json, math, time, random, glob
import numpy as np
import pandas as pd
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import (
    roc_curve, auc, confusion_matrix, classification_report
)

# ---- Paths (chỉnh lại nếu khác) ----
RUNS_DIR   = "runs"          # thư mục cha chứa các run đã train
IMAGES_DIR = "images"        # nơi lưu hình xuất ra
os.makedirs(IMAGES_DIR, exist_ok=True)

# ---- Dataloader/test config (được cell khác dùng) ----
INPUT_SIZE  = 224            # cỡ ảnh (HxW) mà model dùng lúc train
BATCH_SIZE  = 32             # batch size khi evaluate (tùy GPU)

# ---- Device ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---- Seed (tùy chọn) ----
def set_seed(seed=1337):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(1337)

# ===========================================
# Tiện ích: checkpoint, model, thống kê, v.v.
# ===========================================
def pick_checkpoint(run_path: str):
    """
    Trả về đường dẫn checkpoint 'đẹp nhất' trong run:
    ưu tiên: best.* > last.* > *.mtl/*.pt/*.pth ở thư mục checkpoints/
    """
    ckpt_dir = os.path.join(run_path, "checkpoints")
    if not os.path.isdir(ckpt_dir):
        # fallback: tìm ngay trong run_path
        ckpts = sorted(
            [p for p in glob.glob(os.path.join(run_path, "*")) if p.endswith((".mtl",".pt",".pth",".bin",".pth.tar",".ckpt"))]
        )
        return ckpts[-1] if ckpts else None

    # ưu tiên best -> last -> còn lại
    patterns = ["best.*", "last.*", "*.mtl", "*.pt", "*.pth", "*.bin", "*.pth.tar", "*.ckpt"]
    for pat in patterns:
        found = sorted(glob.glob(os.path.join(ckpt_dir, pat)))
        if found:
            return found[-1]
    return None


def _strip_module_prefix(state_dict: dict):
    """Bỏ prefix 'module.' nếu tồn tại (trong trường hợp DDP)."""
    new_sd = {}
    for k, v in state_dict.items():
        if k.startswith("module."): k = k[len("module."):]
        new_sd[k] = v
    return new_sd


def load_checkpoint_to_model(model: nn.Module, ckpt_path: str, device):
    """
    map mọi biến thể checkpoint về state_dict tiêu chuẩn:
    - {'state_dict': ...} hoặc {'model': ...} hoặc chính state_dict
    """
    state = torch.load(ckpt_path, map_location=device)

    if isinstance(state, dict) and "state_dict" in state:
        sd = state["state_dict"]
    elif isinstance(state, dict) and "model" in state:
        sd = state["model"]
    elif isinstance(state, dict):
        sd = state
    else:
        raise RuntimeError("Không nhận diện được định dạng checkpoint.")

    sd = _strip_module_prefix(sd)
    missing, unexpected = model.load_state_dict(sd, strict=False)
    if missing:   print("[ckpt] Missing keys:", list(missing)[:5], "...")
    if unexpected:print("[ckpt] Unexpected keys:", list(unexpected)[:5], "...")
    return model


def count_params(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters())


def file_size_mb(path: str):
    try:
        return os.path.getsize(path) / (1024*1024)
    except Exception:
        return None


# ===========================================
# Auto build model theo tên run (mô-đun bạn có)
# ===========================================
def build_model_auto(run_name: str, num_classes: int):
    """
    Suy ra kiến trúc theo tên run & gọi module tương ứng:
    - 'mobilenetv4'  -> mobilenet_v4.build_model(num_classes)
    - 'efficientnet' -> efficientnet_b0.build_model(num_classes)
    - 'cnn'          -> mtl_cnn.build_model(num_classes)
    Nếu không match, thử lần lượt 3 module; cuối cùng raise lỗi.
    """
    lname = run_name.lower()

    tried = []
    def _try(build_module_name):
        tried.append(build_module_name)
        mod = __import__(build_module_name, fromlist=['*'])
        if hasattr(mod, "build_model"):
            return mod.build_model(num_classes=num_classes)
        # fallback tên hàm khác (nếu bạn đặt khác)
        for cand in ["get_model", "create_model", "make_model"]:
            if hasattr(mod, cand):
                return getattr(mod, cand)(num_classes=num_classes)
        raise AttributeError(f"Module {build_module_name} không có build_model/get_model/...")

    # đoán theo tên
    try:
        if "model.mobilenet_v4" in lname or "mobile" in lname:
            return _try("mobilenet_v4")
        if "model.efficientnet_b0" in lname or "b0" in lname:
            return _try("CustomEfficientNetB0")
        if "model.mtl_cnn" in lname:
            return _try("mtl_cnn")
    except Exception as e:
        print("[auto_model] đoán theo tên thất bại:", e)

    # thử lần lượt
    for m in ["mobilenet_v4", "efficientnet_b0", "mtl_cnn"]:
        try:
            return _try(m)
        except Exception as e:
            print(f"[auto_model] thử {m} lỗi:", e)

    raise RuntimeError(f"Không build được model cho run '{run_name}'. Đã thử: {tried}")


# ===================================================
# FIX Confusion Matrix: class_names theo RUN & remap
# ===================================================
def load_class_names_for_run(run_path: str):
    """
    Lấy class_names đúng thứ tự lúc TRAIN của run này.
    Ưu tiên:
      1) runs/<run>/config.json (keys: 'class_names' | 'idx_to_class' | 'classes')
      2) runs/<run>/label.txt (mỗi dòng 1 lớp)
      3) label.txt(labels.txt) ở project root (fallback)
    """
    # 1) config.json trong run
    cfg_path = os.path.join(run_path, "config.json")
    if os.path.isfile(cfg_path):
        try:
            with open(cfg_path, "r", encoding="utf-8") as f:
                cfg = json.load(f)
            for key in ["class_names", "idx_to_class", "classes"]:
                if key in cfg and cfg[key]:
                    arr = cfg[key]
                    # nếu là dict idx->name thì sort theo idx
                    if isinstance(arr, dict):
                        try:
                            arr = [arr[str(i)] if str(i) in arr else arr[i] for i in sorted(map(int, arr.keys()))]
                        except Exception:
                            # có thể là name->idx, đảo lại
                            name2idx = arr
                            inv = [None]*len(name2idx)
                            for name, idx in name2idx.items():
                                inv[int(idx)] = name
                            arr = inv
                    return list(arr)
        except Exception:
            pass

    # 2) label.txt trong run
    for f in ["label.txt", "labels.txt"]:
        p = os.path.join(run_path, f)
        if os.path.isfile(p):
            with open(p, "r", encoding="utf-8") as g:
                return [line.strip() for line in g if line.strip()]

    # 3) fallback ở project root
    for f in ["label.txt", "labels.txt"]:
        if os.path.isfile(f):
            with open(f, "r", encoding="utf-8") as g:
                return [line.strip() for line in g if line.strip()]

    return None


def build_remap_from_loader_to_run(loader, run_class_names):
    """
    Tạo ánh xạ chỉ số từ dataset.test (loader.dataset.class_to_idx)
    -> index theo run_class_names (đúng thứ tự train).
    """
    if not hasattr(loader.dataset, "class_to_idx"):
        return None
    ds_map = loader.dataset.class_to_idx               # {'Pho': 0, ...} (theo alphabet ImageFolder)
    run_name2idx = {n: i for i, n in enumerate(run_class_names)}
    remap = {}
    for name, src_idx in ds_map.items():
        if name in run_name2idx:
            remap[src_idx] = run_name2idx[name]
    # chỉ chấp nhận nếu mapping đủ “dày”
    return remap if len(remap) >= max(1, int(0.6 * len(run_class_names))) else None


def apply_remap_array(y: np.ndarray, remap: dict | None):
    """Remap mảng nhãn theo dict ánh xạ (nếu có)."""
    if remap is None:
        return y
    return np.array([remap.get(int(t), int(t)) for t in y])


# ===================================================
# (tùy chọn) trích đường dẫn ảnh thật từ DataLoader
# ===================================================
def extract_paths_from_loader(loader):
    """
    Cố gắng lấy list đường dẫn file từ loader.dataset:
    - Với ImageFolder-like: loader.dataset.samples -> [(path, idx), ...]
    - Hoặc loader.dataset.imgs
    Trả về list[str] hoặc None.
    """
    ds = loader.dataset
    if hasattr(ds, "samples") and isinstance(ds.samples, list):
        return [p for p, _ in ds.samples]
    if hasattr(ds, "imgs") and isinstance(ds.imgs, list):
        return [p for p, _ in ds.imgs]
    # cuối cùng trả None (cell phân tích lỗi sẽ tự bỏ qua nếu None)
    return None


Device: cuda


In [156]:
# ==== CELL 1: ROC Curve (OvR) & Macro AUC ====
def plot_roc_ovr(y_true, y_prob, class_names, run_label, max_curves=5, out_dir=IMAGES_DIR):
    C = len(class_names)
    y_true_bin = label_binarize(y_true, classes=np.arange(C))
    fprs, tprs, aucs = [], [], []
    for c in range(C):
        fpr, tpr, _ = roc_curve(y_true_bin[:, c], y_prob[:, c])
        fprs.append(fpr); tprs.append(tpr); aucs.append(auc(fpr, tpr))
    macro_auc = float(np.mean(aucs))
    idx = np.argsort(aucs)[::-1][:max_curves]

    plt.figure(figsize=(7.5,6.5), dpi=160)
    for i in idx:
        plt.plot(fprs[i], tprs[i], lw=2, label=f"{class_names[i]} (AUC={aucs[i]:.3f})")
    plt.plot([0,1],[0,1],'k--', lw=1, label="Random")
    plt.xlim(0,1); plt.ylim(0,1.02)
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
    plt.title(f"ROC Curves – {run_label}\nMacro AUC = {macro_auc:.3f}")
    plt.legend(loc="lower right", fontsize=9, frameon=True)
    out_png = os.path.join(out_dir, f"{run_label}_roc.png")
    plt.tight_layout(); plt.savefig(out_png, dpi=300, bbox_inches="tight"); plt.show()
    print("✓ Saved:", out_png)
    return macro_auc


In [157]:
# ==== CELL 2: Tốc độ & tài nguyên ====
@torch.no_grad()
def benchmark_model(model, device, input_size=(1,3,224,224), repeat=50, warmup=10):
    model.eval().to(device)
    x = torch.randn(*input_size, device=device)
    for _ in range(warmup):
        _ = model(x)
    if device.type == "cuda": torch.cuda.synchronize()
    t0 = time.time()
    for _ in range(repeat):
        _ = model(x)
    if device.type == "cuda": torch.cuda.synchronize()
    t1 = time.time()
    avg = (t1 - t0) / repeat
    return {
        "ms_per_img": avg*1000.0,
        "FPS": 1.0/avg if avg>0 else float("inf")
    }

def plot_speed_table(rows, title="Speed & Resource", out_png=os.path.join(IMAGES_DIR, "speed_summary.png")):
    import pandas as pd
    df = pd.DataFrame(rows).sort_values("FPS", ascending=False)
    display(df)
    fig, ax = plt.subplots(figsize=(10, 0.6 + 0.45*len(df)), dpi=180)
    ax.axis("off")
    tbl = ax.table(
        cellText=df[["run","FPS","ms_per_img","params(M)","size(MB)"]]
                .assign(FPS=lambda d: d["FPS"].map(lambda x: f"{x:.1f}"),
                        **{"ms_per_img":lambda d: d["ms_per_img"].map(lambda x: f"{x:.2f}")})
                .values,
        colLabels=["Model","FPS (↑)","ms/img (↓)","Params","Size"],
        loc="center", cellLoc="center"
    )
    tbl.auto_set_font_size(False); tbl.set_fontsize(9); tbl.scale(1,1.1)
    plt.title(title); plt.tight_layout(); plt.savefig(out_png, dpi=300, bbox_inches="tight"); plt.show()
    print("✓ Saved:", out_png)


In [158]:
# ==== CELL 3: Biểu đồ Loss/Accuracy theo epoch từ history.* ====
import pandas as pd

def _pick_history_path(run_path):
    for f in ["history.csv", "train_log.csv", "metrics.csv", "history.json"]:
        p = os.path.join(run_path, f)
        if os.path.isfile(p): return p
    return None

def _pick_col(df, pats):
    pats = [p.lower() for p in pats]
    for c in df.columns:
        cl = c.lower().strip()
        if any(re.search(p, cl) for p in pats):
            return c
    return None

def plot_history_from_run(run_path, run_label):
    hist = _pick_history_path(run_path)
    if hist is None:
        print(f"[{run_label}] Không tìm thấy history.csv/json.")
        return None
    df = pd.read_json(hist) if hist.endswith(".json") else pd.read_csv(hist)
    df = df.copy()
    # epoch
    c_epoch = _pick_col(df, [r"^epoch$", r"^epochs?$"])
    epoch = df[c_epoch].to_numpy() if c_epoch else np.arange(1, len(df)+1)
    # keys
    c_tr_loss = _pick_col(df, [r"^loss$", r"train.*loss"])
    c_va_loss = _pick_col(df, [r"val.*loss", r"valid.*loss"])
    c_tr_acc  = _pick_col(df, [r"^acc$", r"train.*acc", r"train.*accuracy", r"accuracy$"])
    c_va_acc  = _pick_col(df, [r"val.*acc", r"valid.*acc", r"val.*accuracy", r"valid.*accuracy"])

    fig, ax = plt.subplots(1,2, figsize=(12,4.5), dpi=160)
    if c_tr_loss: ax[0].plot(epoch, df[c_tr_loss], label="Train loss")
    if c_va_loss: ax[0].plot(epoch, df[c_va_loss], label="Val loss")
    ax[0].set_title(f"{run_label} – Loss"); ax[0].set_xlabel("Epoch"); ax[0].set_ylabel("Loss"); ax[0].legend()

    if c_tr_acc: ax[1].plot(epoch, df[c_tr_acc], label="Train acc")
    if c_va_acc: ax[1].plot(epoch, df[c_va_acc], label="Val acc")
    ax[1].set_title(f"{run_label} – Accuracy"); ax[1].set_xlabel("Epoch"); ax[1].set_ylabel("Acc"); ax[1].legend()

    out_png = os.path.join(IMAGES_DIR, f"{run_label}_history.png")
    plt.tight_layout(); plt.savefig(out_png, dpi=300, bbox_inches="tight"); plt.show()
    print("✓ Saved:", out_png)
    return out_png


In [159]:
# ==== CELL 4: Confusion Matrix (chuẩn hoá theo hàng) ====
def plot_confusion_matrix(y_true, y_pred, class_names, run_label, threshold=0.10, out_dir=IMAGES_DIR):
    cm_counts = confusion_matrix(y_true, y_pred, labels=range(len(class_names)))
    cm = cm_counts.astype(float) / cm_counts.sum(axis=1, keepdims=True)
    cm = np.nan_to_num(cm)
    fig, ax = plt.subplots(figsize=(14,11), dpi=160)
    sns.heatmap(cm, vmin=0, vmax=1, cmap="Blues", square=True,
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'shrink': .6}, ax=ax)
    # annotate
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            show = (i==j) or (cm[i,j] >= threshold)
            if show and cm_counts[i,j] > 0:
                ax.text(j+0.5, i+0.5, f"{cm[i,j]*100:.0f}%\n({cm_counts[i,j]})",
                        ha="center", va="center", fontsize=7, color="black")
    ax.set_xlabel("Predicted"); ax.set_ylabel("True")
    ax.set_title(f"Confusion Matrix (row-norm) – {run_label}")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right", fontsize=8)
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=8)
    out_png = os.path.join(out_dir, f"{run_label}_cm.png")
    plt.tight_layout(); plt.savefig(out_png, dpi=300, bbox_inches="tight"); plt.show()
    print("✓ Saved:", out_png)
    return out_png


In [160]:
# ==== CELL 5: Top-20 cặp dễ nhầm nhất ====
def plot_top_confusions(y_true, y_pred, class_names, topk=20, out_dir=IMAGES_DIR, run_label="model"):
    cm = confusion_matrix(y_true, y_pred, labels=range(len(class_names)))
    row_sum = cm.sum(axis=1, keepdims=True).clip(min=1)
    cmn = cm / row_sum

    pairs = []
    for i in range(cmn.shape[0]):
        for j in range(cmn.shape[1]):
            if i==j: continue
            pairs.append((i,j, cmn[i,j], cm[i,j]))  # (true, pred, rate, count)
    pairs.sort(key=lambda x: x[2], reverse=True)
    pairs = pairs[:topk]

    import pandas as pd
    df = pd.DataFrame({
        "True → Pred": [f"{class_names[i]} → {class_names[j]}" for i,j,_,_ in pairs],
        "Nhầm (%)":   [round(r*100,2) for _,_,r,_ in pairs],
        "Số ảnh":     [int(c) for *_,c in pairs],
    })
    display(df)

    # Bar chart
    fig, ax = plt.subplots(figsize=(11, 0.5*len(df)+1.5), dpi=160)
    ax.barh(df["True → Pred"][::-1], df["Nhầm (%)"][::-1])
    ax.set_xlabel("Tỉ lệ nhầm (%)"); ax.set_title(f"Top-{topk} cặp dễ nhầm – {run_label}")
    plt.tight_layout()
    out_png = os.path.join(out_dir, f"{run_label}_top{topk}_confusions.png")
    plt.savefig(out_png, dpi=300, bbox_inches="tight"); plt.show()
    print("✓ Saved:", out_png)
    return out_png, df


In [161]:
# ==== CELL 6: Phân tích lỗi – ảnh sai & ví dụ theo cặp ====
@torch.no_grad()
def collect_preds_with_paths(model, loader, device):
    y_true, y_pred, y_prob = [], [], []
    paths = extract_paths_from_loader(loader)
    n_seen = 0
    for x, y in loader:
        x = x.to(device, non_blocking=True)
        prob = F.softmax(model(x), dim=1).cpu().numpy()
        pred = prob.argmax(1)
        y_true.append(y.numpy()); y_pred.append(pred); y_prob.append(prob.max(1))
        n_seen += y.shape[0]
    y_true = np.concatenate(y_true); y_pred = np.concatenate(y_pred); y_prob = np.concatenate(y_prob)
    if len(paths) != n_seen: paths = paths[:n_seen]
    return y_true, y_pred, y_prob, paths

def plot_misclassified_grid(run_label, class_names, y_true, y_pred, y_prob, paths,
                            top_k=30, cols=5, out_dir=IMAGES_DIR):
    os.makedirs(out_dir, exist_ok=True)
    wrong = np.where(y_true != y_pred)[0]
    if wrong.size == 0:
        print(f"[{run_label}] ✅ Không có lỗi."); return None
    order = wrong[np.argsort(-y_prob[wrong])][:top_k]
    rows = math.ceil(len(order)/cols)
    fig, axes = plt.subplots(rows, cols, figsize=(cols*4, rows*4), dpi=150)
    axes = np.array(axes).reshape(rows, cols)
    for ax in axes.ravel(): ax.axis("off")
    for i, idx in enumerate(order):
        r, c = divmod(i, cols); ax = axes[r,c]
        try: img = Image.open(paths[idx]).convert("RGB")
        except: continue
        ax.imshow(img)
        t = class_names[y_true[idx]]; p = class_names[y_pred[idx]]; conf = y_prob[idx]
        ax.set_title(f"True: {t}\nPred: {p} ({conf:.2f})", fontsize=9)
        ax.axis("off")
    fig.suptitle(f"Ảnh dự đoán sai (Top-{len(order)}) – {run_label}", fontsize=13)
    out_png = os.path.join(out_dir, f"{run_label}_mis_top{len(order)}.png")
    plt.tight_layout(); plt.savefig(out_png, dpi=300, bbox_inches="tight"); plt.show()
    print("✓ Saved:", out_png)
    return out_png


In [162]:
def evaluate_one_run(run_path: str):
    run_name = os.path.basename(run_path.rstrip(os.sep))
    print(f"▶️ Evaluate: {run_name}")

    # 1) checkpoint & model
    ckpt = pick_checkpoint(run_path)
    if ckpt is None:
        print(f"  ⚠️ Không thấy checkpoint trong {run_name}."); 
        return None

    # 2) test loader
    loader, names_from_ds = make_test_loader_safe()

    # 3) LẤY CLASS NAMES ĐÚNG THỨ TỰ TRAIN CHO RUN NÀY
    run_class_names = load_class_names_for_run(run_path)
    if run_class_names is None:
        # fallback: nếu bạn đã gán CLASS_NAMES ở Cell khác thì dùng; nếu không dùng từ dataset
        run_class_names = CLASS_NAMES or names_from_ds
    if run_class_names is None:
        raise RuntimeError("Không xác định được class_names cho run này.")

    # 4) build & load model đúng num_classes theo run
    model = build_model_auto(run_name, num_classes=len(run_class_names))
    model = load_checkpoint_to_model(model, ckpt, device).to(device).eval()

    # 5) thu thập logits
    y_true, y_pred, y_prob = collect_logits(model, loader, device)

    # 6) remap y_true từ không gian của dataset.test -> không gian NHÃN THEO RUN
    remap = build_remap_from_loader_to_run(loader, run_class_names)
    if remap is None:
        print("  ⚠️ Không tạo được remap đáng tin giữa dataset.test và nhãn theo run. "
              "Giả định cùng thứ tự. Nếu CM vẫn sai cột, hãy kiểm tra label order.")
    y_true = apply_remap_array(y_true, remap)

    # 7) VẼ: ROC, CM, TOP CONFUSIONS, MIS-IMAGES
    macro_auc = plot_roc_ovr(y_true, y_prob, run_class_names, run_label=run_name, max_curves=5)
    _ = plot_confusion_matrix(y_true, y_pred, run_class_names, run_label=run_name, threshold=0.10)
    _ = plot_top_confusions(y_true, y_pred, run_class_names, topk=20, run_label=run_name)
    _ = plot_misclassified_grid(
            run_name, run_class_names, y_true, y_pred, y_prob.max(1), extract_paths_from_loader(loader))

    # 8) Loss/Acc theo epoch (nếu có)
    _ = plot_history_from_run(run_path, run_name)

    # 9) Tốc độ & tài nguyên
    speed = benchmark_model(model, device, input_size=(1,3,INPUT_SIZE,INPUT_SIZE), repeat=60, warmup=12)
    params_m = count_params(model)/1e6
    size_mb  = file_size_mb(ckpt) or 0.0

    result = {
        "run": run_name,
        "macro_auc": float(macro_auc),
        "acc": float((y_true==y_pred).mean()),
        "params(M)": round(params_m, 2),
        "size(MB)": round(size_mb, 1),
        "FPS": round(speed["FPS"], 1),
        "ms_per_img": round(speed["ms_per_img"], 2)
    }
    print("— Summary:", result)
    return result


In [163]:
# ==== CELL 8: Đánh giá tất cả runs & vẽ bảng tốc độ/tài nguyên ====
import pandas as pd

def evaluate_all_runs(runs_dir=RUNS_DIR):
    if not os.path.isdir(runs_dir):
        print("⚠️ RUNS_DIR không tồn tại:", runs_dir); return None
    rows = []
    for run_name in sorted(os.listdir(runs_dir)):
        run_path = os.path.join(runs_dir, run_name)
        if not os.path.isdir(run_path): continue
        try:
            res = evaluate_one_run(run_path)
            if res: rows.append(res)
        except Exception as e:
            print(f"[warn] Lỗi {run_name}: {e}")
    if not rows:
        print("⚠️ Không có kết quả."); return None

    df = pd.DataFrame(rows).sort_values("macro_auc", ascending=False)
    display(df)

    # Bảng tốc độ & tài nguyên
    plot_speed_table(
        rows=[{"run":r["run"], "FPS":r["FPS"], "ms_per_img":r["ms_per_img"],
               "params(M)":f'{r["params(M)"]:.2f}M', "size(MB)":f'{r["size(MB)"]:.1f} MB'}
              for r in rows],
        title="Speed & Resource Summary"
    )
    return df


In [164]:
# ==== CELL 9: GỌI CHẠY ====

# Cách 1: Chạy 1 mô hình (điền đúng tên thư mục run)
# _ = evaluate_one_run(os.path.join(RUNS_DIR, "mtl-efficientnet_b0-20251029-233246"))

# Cách 2: Chạy tất cả mô hình trong RUNS_DIR
_ = evaluate_all_runs(RUNS_DIR)


▶️ Evaluate: mtl-cnn
[auto_model] thử mobilenet_v4 lỗi: No module named 'mobilenet_v4'
[auto_model] thử efficientnet_b0 lỗi: No module named 'efficientnet_b0'
[auto_model] thử mtl_cnn lỗi: No module named 'mtl_cnn'
[warn] Lỗi mtl-cnn: Không build được model cho run 'mtl-cnn'. Đã thử: ['mobilenet_v4', 'efficientnet_b0', 'mtl_cnn']
▶️ Evaluate: mtl-efficientnet_b0
[auto_model] đoán theo tên thất bại: No module named 'CustomEfficientNetB0'
[auto_model] thử mobilenet_v4 lỗi: No module named 'mobilenet_v4'
[auto_model] thử efficientnet_b0 lỗi: No module named 'efficientnet_b0'
[auto_model] thử mtl_cnn lỗi: No module named 'mtl_cnn'
[warn] Lỗi mtl-efficientnet_b0: Không build được model cho run 'mtl-efficientnet_b0'. Đã thử: ['CustomEfficientNetB0', 'mobilenet_v4', 'efficientnet_b0', 'mtl_cnn']
▶️ Evaluate: mtl-mobilenetv4
[auto_model] đoán theo tên thất bại: No module named 'mobilenet_v4'
[auto_model] thử mobilenet_v4 lỗi: No module named 'mobilenet_v4'
[auto_model] thử efficientnet_b0 lỗi: